In [21]:
#Cargar la extensión autoreload
%load_ext autoreload
%autoreload 2
# Retriever tools
from libs.chatbot import Chatbot
from libs.vector_retriever import VectorRetriever
from libs.lexical_retriever import LexicalRetriever
from libs.basic_document_storage import BasicStorage
from libs.hybrid_retriever import HybridRetriever
from libs.pdf_loader import PDFLoader
from libs.basic_text_splitter import BasicTextSplitter

# Prompt Tools
from prompts.academic_assistant_prompt import system_prompt as academic_system_prompt
from prompts.academic_assistant_prompt import user_prompt as academic_user_prompt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
# Load the documents
pdf_loader = PDFLoader(file_filter={".pdf"})
documents = pdf_loader.load_data()
# Split the documents in chunks
splitter = BasicTextSplitter()
chunked_documents = splitter.split_documents(documents)
# Store the documents
basic_storage = BasicStorage()
basic_storage.add_documents(chunked_documents)


In [3]:
# 2. Create a retriever to handle the retrieval process.
# For handling the storage and retrieval of data chunks with semantic search
semantic_retriever = VectorRetriever()
# For handling the storage and retrieval of data chunks with lexical search
lexical_retriever = LexicalRetriever()
# A retriever for handling all the retrieval process
hybrid_retriever = HybridRetriever(storage_= basic_storage, retrievers=[semantic_retriever,lexical_retriever])
# Add the documents to the retriever
print("Adding documents to the retriever")
hybrid_retriever.add(chunked_documents)


Adding documents to the retriever


WARNING clustering 659 points to 25 centroids: please provide at least 975 training points


BM25S Create Vocab:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/659 [00:00<?, ?it/s]

In [36]:
bot = Chatbot(hybrid_retriever,
              system_prompt=academic_system_prompt,
              user_prompt=academic_user_prompt
              )

In [5]:
query1 = "Cuales son las distintas formas de turismo"
query2= "Hablame de los pasos principales de la organización y el desarrollo de un viaje y los tipos de entidades participantes en ello"

In [37]:
r1 =bot.reply(query1)
print(str(r1))

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

ChatCompletionOutputMessage(role='assistant', content='{\n    "context": "Turismo de sol y playa ( relax + activo). Turismo cultural (ciudades, entornos culturales significativos). Turismo de reuniones (congresos, convenciones y otros eventos). Turismo deportivo (activo: torneos + competencias: pasivo: espectadores). Turismo náutico (buceo, vida a bordo, paseos en embarcaciones, yatismo, veleros, etcétera). Turismo académico y científico (cursos, idiomas, etcétera). Turismo de naturaleza (ecoturismo, agroturismo, vacaciones en la naturaleza). Turismo de salud (belleza, salud mental y corporal, hospitalario y no hospitalario). Turismo de negocios. Viajes de incentivo. Turismo de cruceros. Turismo de intereses sociales. Turismo de intercambio profesional. Turismo religioso. Turismo de hobbies, aficiones. Turismo de bodas y lunas de miel. Turismo de aventura (hard y soft). Turismo de parques temáticos (sitios de diversión o aprendizaje o ambos a la vez, normalmente antrópicos).",\n    "qu

In [39]:
from json import loads

In [38]:
r2 =bot.reply(query2)
print(str(r2))

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

ChatCompletionOutputMessage(role='assistant', content='{\n    "context": "En la fase de preparación del viaje y desplazamiento:\n    \x81 Fenómenos de comunicación social (en cuanto a la imagen de un país o región en otro) y comunicación promocional: de empuje (atracción) del destino o producto turístico específico hacia (de) la clientela potencial. Estas son relaciones del tipo psicosociológicas.\n    \x81 Relaciones comerciales interempresas (contratos previos entre suministradores de servicios de viajes y estancias del destino con entidades del emisor) y relaciones comerciales de compra-venta de viajes, entre los distintos tipos de clientes finales y las agencias o vendedores de viajes.\n    \x81 Relaciones y normativas jurídicas: de los procesos migratorios o de salida-entrada entre países (requisitos de pasaportes o de otras identi- ficaciones, visados, regulaciones sanitarias, etcétera) y de los procesos aduanales o de control de ingreso de salidas-entradas de mercancías entre pa

In [40]:
json_o = loads(str(r2))
print(json_o['response'])

JSONDecodeError: Invalid control character at: line 2 column 70 (char 71)

In [8]:
bot.history('all')

[{'role': 'assistant',
  'content': "I'm working on a project that involves creating a simple chatbot for a customer service department. The chatbot will be used to handle common customer inquiries, freeing up human agents to focus on more complex issues. Here's a list of common questions that the chatbot should be able to handle:\n\n1. What are your hours of operation?\n2. How can I track my order?\n3. Where is my nearest store location?\n4. What are your return policies?\n5. How do I reset my password?\n6. Can I get a refund or exchange an item?\n7. How do I contact customer service?\n8. What are your shipping policies?\n9. How do I cancel an order?\n10. What are your payment options?\n\nI'd like to create a simple decision tree to help the chatbot respond appropriately to these questions. Can you help me create a basic decision tree for these common questions?\n\nHere's a basic structure for the decision tree:\n\n```\nRoot\n  |\n  |--- Question 1: What are your hours of operation?\n